In [ ]:
from pathlib import Path
import tifffile
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pyclesperanto_prototype as cle
from utils_cellpose import list_images, read_image, segment_nuclei_2d

cle.select_device("RTX")

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [ ]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Construct ROI path from directory_path above
roi_path = directory_path / "ROIs"

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

<h3>Define your nuclei channel and your nuclear segmentation parameters</h3>

Modify the values for <code>slicing factor</code>, <code>nuclei_channel</code>, <code>cellpose_nuclei_diameter</code> and <code>gaussian_sigma</code>


In [17]:
# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 2

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "2D"

# This is a placeholder to later choose from fine-tuned Cellpose models (default nuclei model in Cellpose 3.0)
model_name = "Cellpose"

# Define your nuclei diameter, it speeds up nuclei detection, if unknown leave it as None
cellpose_nuclei_diameter = None

# Define the amount of blur applied to nuclei
# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 0

<h3>Predict nuclei labels and store them as .tiff files</h3>

In [ ]:
# List of subfolder names
try: 
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]
    print(f"The following regions of interest will be analyzed: {roi_names}")

except FileNotFoundError:
    roi_names = ["full_image"]

for image in tqdm(images):

    for roi_name in roi_names:
        # Check if the prediction has already been generated
        try:
            file_path = Path(image)
            filename = file_path.stem
            tifffile.imread(directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name / f"{filename}.tiff")
            print(f"\nWARNING: Nuclei predictions already found for: {filename} ROI: {roi_name}")
            print("Make sure nuclei labels were generated using the same settings.")

        except FileNotFoundError:

            # Generate maximum intensity projection and extract filename
            img_mip, filename = read_image(image, slicing_factor)

            # Slice the nuclei stack
            nuclei_img = img_mip[nuclei_channel, :, :]
            
            for roi_name in roi_names:
            
                print(f"Generating nuclei predictions for {roi_name} ROI")

                # Construct path to read ROI
                roi_path = directory_path / "ROIs" / roi_name / f"{filename}.tiff"

                try:

                    # Read the .tiff files containing the user-defined ROIs
                    roi = tifffile.imread(roi_path)

                    # We will create a mask where roi is greater than or equal to 1
                    mask = (roi >= 1).astype(np.uint8)

                    # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
                    masked_nuclei_img = np.where(mask, nuclei_img, 0)

                except FileNotFoundError:
                    # If no ROI is saved the script will predict nuclei in the entire nuclei_img input
                    masked_nuclei_img = nuclei_img

                # Segment nuclei and return labels
                nuclei_labels = segment_nuclei_2d(masked_nuclei_img, gaussian_sigma, cellpose_nuclei_diameter)

                # Save nuclei labels as .tiff files to reuse them later
                # Create nuclei_predictions directory if it does not exist
                try:
                    os.makedirs(directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name)
                except FileExistsError:
                    pass

                # Construct path to store
                nuclei_preds_path = directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name / f"{filename}.tiff"

                # Save mask (binary image)
                tifffile.imwrite(nuclei_preds_path, nuclei_labels)

print("\nNuclei prediction completed")